How to use the TileLevelDataset
-------------------------------

In [1]:
import os
import time
import glob
import pandas as pd
import slide_tools

In [2]:
root = "/mnt/data/Lennard/gyn"
csv_train = "ago-tr1/csv/finetune_train_1.csv"
frame = pd.read_csv(os.path.join(root, csv_train))
frame

,SlideNr,HRD(BRCA1),label,slide,annotation
0,10,0,ago-tr1/labels/10.json,ago-tr1/slides/10.svs,ago-tr1/annotations/10.geojson
1,100,1,ago-tr1/labels/100.json,ago-tr1/slides/100.svs,ago-tr1/annotations/100.geojson
2,101,1,ago-tr1/labels/101.json,ago-tr1/slides/101.svs,ago-tr1/annotations/101.geojson
3,103,1,ago-tr1/labels/103.json,ago-tr1/slides/103.svs,ago-tr1/annotations/103.geojson
4,106,1,ago-tr1/labels/106.json,ago-tr1/slides/106.svs,ago-tr1/annotations/106.geojson
...,...,...,...,...,...
127,84,1,ago-tr1/labels/84.json,ago-tr1/slides/84.svs,ago-tr1/annotations/84.geojson
128,85,1,ago-tr1/labels/85.json,ago-tr1/slides/85.svs,ago-tr1/annotations/85.geojson
129,9,0,ago-tr1/labels/9.json,ago-tr1/slides/9.svs,ago-tr1/annotations/9.geojson
130,92,1,ago-tr1/labels/92.json,ago-tr1/slides/92.svs,ago-tr1/annotations/92.geojson


### Create dataset from slide, annotations, tile labels and slide labels

In [3]:
rootify = lambda path: os.path.join(root, path)

In [4]:
%%time
ds = slide_tools.tile_level.TileLevelDataset(
    slide_paths=frame.slide.apply(rootify),
    annotation_paths=frame.annotation.apply(rootify),
    label_paths=frame.label.apply(rootify),
    global_labels=frame[["HRD(BRCA1)"]].to_dict("records"),
    simplify_tolerance=100,
)
print(f"Found {len(ds.samples)} tiles of native size")

# Found 16101629 tiles of native size
# CPU times: user 1min 5s, sys: 2.82 s, total: 1min 8s
# Wall time: 1min 8s

[Plugin: cucim.kit.cuslide] Loading the dynamic library from: /home/caduser/anaconda3/envs/tmmae/lib/python3.8/site-packages/cucim/clara/cucim.kit.cuslide@22.02.00.so
Initializing plugin: cucim.kit.cuslide (interfaces: [cucim::io::IImageFormat v0.1]) (impl: cucim.kit.cuslide)
[Plugin: cucim.kit.cumed] Loading the dynamic library from: /home/caduser/anaconda3/envs/tmmae/lib/python3.8/site-packages/cucim/clara/cucim.kit.cumed@22.02.00.so
Initializing plugin: cucim.kit.cumed (interfaces: [cucim::io::IImageFormat v0.1]) (impl: cucim.kit.cumed)


Found 16101629 tiles of native size
CPU times: user 1min 5s, sys: 2.82 s, total: 1min 8s
Wall time: 1min 8s


### Tiles with overlap and centroid within annotation

In [5]:
%%time
region_overlap = 0.5
ds.setup_regions(centroid_in_annotation=True, region_overlap=region_overlap, with_labels=True)
ds.setup_epoch()
print(f"Found {len(ds.samples)} tiles of native size inside annotations with overlap of {region_overlap}")

# Found 28201545 tiles of native size inside annotations with overlap of 0.5
# CPU times: user 36.9 s, sys: 5.08 s, total: 42 s
# Wall time: 42.1 s

Found 28201545 tiles of native size inside annotations with overlap of 0.5
CPU times: user 36.9 s, sys: 5.08 s, total: 42 s
Wall time: 42.1 s


### Balancing by size 

In [6]:
%%time
ds.setup_epoch(
    shuffle=True,
    balance_size_by=slide_tools.objects.BalanceMode.MIN,  # MIN=Undersampling, MAX=Oversampling, MEDIAN/MEAN=Mix
    balance_label_key="HRD(BRCA1)",
    balance_label_bins=2,
    shuffle_chunk_size=16
)
print(f"Found {len(ds.samples)} tiles of native size inside annotations with overlap of {region_overlap} and balanced by size and label")

# Found 1733028 tiles of native size inside annotations with overlap of 0.5 and balanced by size and label
# CPU times: user 4.74 s, sys: 2.51 s, total: 7.25 s
# Wall time: 7.29 s

Found 1733028 tiles of native size inside annotations with overlap of 0.5 and balanced by size and label
CPU times: user 4.74 s, sys: 2.51 s, total: 7.25 s
Wall time: 7.29 s


### Let's have a look at the speed

In [7]:
import torch

In [8]:
bs = 128
num_workers = 8
dl = torch.utils.data.DataLoader(ds, shuffle=False, batch_size=bs, num_workers=num_workers)

In [9]:
N = 100
for i, batch in enumerate(dl):
    if i == 20:
        t0 = time.time()
    if i == (N + 20):
        t = time.time() - t0
        break
        
print(f"{bs*N/t:.0f} 240x240 tiles per second with {num_workers=}")

# 1139 240x240 tiles per second with num_workers=8

1139 240x240 tiles per second with num_workers=8


### Checkout the main documentation for further options

In [10]:
slide_tools.tile_level.TileLevelDataset?

Init signature: slide_tools.tile_level.TileLevelDataset(*args, **kwds)
Docstring:     
An abstract class representing a :class:`Dataset`.

All datasets that represent a map from keys to data samples should subclass
it. All subclasses should overwrite :meth:`__getitem__`, supporting fetching a
data sample for a given key. Subclasses could also optionally overwrite
:meth:`__len__`, which is expected to return the size of the dataset by many
:class:`~torch.utils.data.Sampler` implementations and the default options
of :class:`~torch.utils.data.DataLoader`.

.. note::
  :class:`~torch.utils.data.DataLoader` by default constructs a index
  sampler that yields integral indices.  To make it work with a map-style
  dataset with non-integral indices/keys, a custom sampler must be provided.
Init docstring:
Map-style dataset for tile-level training of WSI.

The idea is to call .setup_epoch(...) at the beginning of every epoch to sample
new tiles from the pool of available WSI. This will populate 

In [11]:
ds.setup_regions?

Signature:
ds.setup_regions(
    size: Union[Sequence[int], NoneType] = None,
    unit: slide_tools.objects.constants.SizeUnit = <SizeUnit.PIXEL: 'pixel'>,
    level: int = 0,
    centroid_in_annotation: bool = False,
    annotation_align: bool = False,
    region_overlap: float = 0.0,
    with_labels: bool = False,
    filter_by_label_func: Union[Callable, NoneType] = None,
)
Docstring:
Call .setup_regions(...) for all .slides
See `slide_tools.objects.Slide`
File:      ~/Lennard/slide_tools/slide_tools/tile_level/tile_level_dataset.py
Type:      method


In [12]:
ds.setup_epoch?

Signature:
ds.setup_epoch(
    balance_size_by: Union[slide_tools.objects.constants.BalanceMode, int, NoneType] = None,
    balance_label_key: Union[str, NoneType] = None,
    balance_label_bins: int = 10,
    shuffle: bool = False,
    shuffle_chunk_size: int = 1,
)
Docstring:
Populate .samples with corresponding region from all .slides to iterate over.

Args:
    balance_size_by (BalanceMode or int, optional): Determines N_samples = len(.slides) * balance_size_by
    balance_label_key (str, optional): label used for balancing (will be digitized, see np.digitize)
    balance_label_bins (int, optional): number of bins for label digitization (default: 10)
    shuffle (bool): shuffle samples or not
    shuffle_chunk_size (int): chunk samples before shuffling for faster loading (default: 1)

Balancing by size and/or label will determine a regions weight for being sampled with replacement.
Shuffling can be done in chunks so that a worker is more likely to read multiple (neighboring) tiles
